### SMS Spam classification

In [13]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv'


In [14]:
df = pd.read_csv(url, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


- Preprocessing

In [15]:
# Selection
df = df[['v1', 'v2']]

In [16]:
# 결측치 확인
df.isna().sum().sum()

0

In [17]:
# 중복 데이터 확인
print(df.shape, df.v2.nunique())

(5572, 2) 5169


In [18]:
# 중복 데이터 제거
df.drop_duplicates(subset=['v2'], inplace=True)
df.shape


(5169, 2)

In [19]:
# Label Encoding, ['ham', 'spam'] --> [0, 1]
df.v1 = df.v1.replace(['ham', 'spam'], [0, 1])
df.head(3)

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.v2.values, df.v1.values, stratify=df.v1.values, test_size=0.2, random_state=2023
)

- Text Encoding

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')


In [22]:
# ❌Never do that!!❌
# different vocabulary in train and test
X_train_cv = cvect.fit_transform(X_train)
X_test_cv = cvect.fit_transform(X_test)

In [23]:
# 변환된 단어 세트가 train/test dataset에서 동일해야 함
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((4135, 7336), (1034, 7336))

- 학습 및 평가

In [24]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
lrc.fit(X_train_cv, y_train)

In [ ]:
lrc.score(X_test_cv, y_test)

- Bigram

In [ ]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape


In [ ]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_cv2, y_train)
lrc.score(X_test_cv2, y_test)